In [ ]:
import numpy as np
import scipy as sp

In [ ]:
from quadrotor.dynamics import QuadrotorDynamicsBase, QuadrotorState
from quadrotor.controller import ControllerBase, QuadrotorCommands
from quadrotor.trajectory import TrajectoryBase, TrajectoryState

In [ ]:
from sym import Rot3

def dRot3(R: Rot3, omega: np.ndarray) -> Rot3:
    """
    NOTE(roy): If interested, we can discuss the math here later. There are other
    ways of doing this but to keep the ode_int formulation, we specifically require
    dq/dt. Refer to 'Baseile Graf - Quaternions and Dynamics' or (for intuition)
    https://math.stackexchange.com/questions/1896379/how-to-use-the-quaternion-derivative
    """
    quat = np.array(R.to_storage())  # Convert to raw quaternion
    (q0, q1, q2, q3) = quat  # xyzw storage, see https://symforce.org/api-gen-cpp/class/classsym_1_1Rot3.html

    G = np.array([[ q3,  q2, -q1, -q0],
                [-q2,  q3,  q0, -q1],
                [ q1, -q0,  q3, -q2]])
    quat_dot = (G.T @ omega) / 2
    return Rot3.from_storage(quat_dot)

In [ ]:
from dataclasses import dataclass, field

@dataclass
class Params:
    """A little utility class to hold our quadrotor parameters"""
    mass: float = 1.352  # [kg]

    # NOTE(roy) this is a matrix now as discussed!
    # NOTE(roy) don't get scared by the field/default_factory thing! It's just a way to do default field initialization for dataclasses,
    # see https://docs.python.org/3/library/dataclasses.html. Don't worry about it!
    inertia: np.ndarray = field(default_factory=lambda: np.array([
        [9.8e-3, 0, 0],
        [0, 10.02e-3, 0],
        [0, 0, 18.6e-3],
    ]))

    rotor_diameter: float = 10 * 0.0254  # [m] 10 inches

    static_thrust_coefficient: float = 0.14553  # [-] Thrust coefficient
    static_torque_coefficient: float = 0.01047  # [-] Torque coefficient

    # NOTE(roy) assume this is the same front-to-back and left-to-right
    arm_length: float = 0.3814 / 2.0  # [m] Arm length

    # Universal constants
    g: float = 9.80665  # [m/s2] Gravity
    rho: float = 1.225  # [kg/m3] Sea-level density

    def rotor_model(self, static_coefficient: float) -> float:
        # NOTE(roy) I'm just including this for completeness sake, this means we can e.g. see how air density
        # influences our simulation. Refer to this source for derivation:
        # https://web.mit.edu/16.unified/www/FALL/thermodynamics/notes/node86.html
        return self.rho * static_coefficient * (self.rotor_diameter ** 4) / (4 * np.pi ** 2)

    # NOTE(roy) if you're unsure what @property decorators do: they make it so that we can call k_thrust as if it was just a class
    # variable. I.e. we'd write params.k_thrust now, rather than params.k_thrust()
    @property
    def k_thrust(self):
        return self.rotor_model(self.static_thrust_coefficient)

    @property
    def k_torque(self):
        return self.rotor_model(self.static_torque_coefficient)

# Simply treat as a global
p = Params()

### The dynamics

In [1]:
# Copy your dynamics here!
raise NotImplementedError("Dynamics not implemented!")

NotImplementedError: Dynamics not implemented!

### The controller

In [ ]:
@dataclass
class ControllerParams:
    """The same as above but for the controller gains"""

    # NOTE(roy) diag just means we get a matrix with these values on the diagonal,
    # so these are 3x3 matrices (print them if you're unsure!)
    # We use this formulation because it makes the math a bit more readable later on!
    K_p: np.ndarray = field(default_factory=lambda: np.diag([1, 1, 100]))
    K_d: np.ndarray = field(default_factory=lambda: np.diag([0.5, 0.5, 10]))

    # Added gains for the attitude controller
    Ka_p: np.ndarray = field(default_factory=lambda: np.diag([300, 300, 50]))
    Ka_d: np.ndarray = field(default_factory=lambda: np.diag([50, 50, 20]))

    # Min/max rotor rates!
    rotor_rate_min: float = 180
    rotor_rate_max: float = 600

controller_p = ControllerParams()

In [ ]:
# Copy your controller here!
raise NotImplementedError("Controller not implemented!")

### The trajectory

In this notebook, you'll take your working controller and start generating some fun trajectories for it! I've provided a small `Waypoint` class which describes a single point for the quadrotor to visit. Then I've also provided a `WaypointTrajectory` class that takes in a list of those waypoints. I already wrote some of the code such that for each time `t` we know which waypoint comes before and after it.

It's your job to generate smooth (and fast!) trajectories between the waypoints. What I would do to start:
1. Generate the simplest possible trajectory and just **interpolate** the position between the previous and next waypoint.
2. Generate a bang-coast-bang trajectory with a given maximum velocity- and acceleration
3. (If you get here you're already doing great!)
4. Generate a smooth polynomial trajectory! This is tricky and I don't expect you to get to this at all - you will also need to precompute the trajectory if you follow the method from the slides. Happy to help if you do have questions about this!

In [2]:
@dataclass
class Waypoint:
    time: float  # Time to arrive at waypoint
    
    position: np.ndarray  # xyz-position
    yaw: float = 0.0

    speed: np.ndarray = field(default_factory=lambda: np.zeros(3))  # xyz-velocity

NameError: name 'dataclass' is not defined

In [3]:
import typing as T

class WaypointTrajectory(TrajectoryBase):
    def __init__(self, waypoints: T.List[Waypoint]) -> None:
        self.waypoints = waypoints
        assert self.waypoints[0].time == 0, "First waypoint has to have time 0!"

        # Doublecheck the waypoints are in order:
        for a, b in zip(self.waypoints[:-1], self.waypoints[1:]):
            assert b.time > a.time, "Waypoints time needs to be increasing!"
    
    def eval(self, t: float):
        # If we're at the last waypoint just pick the last way point
        next_waypoint = next((w for w in self.waypoints if w.time >= t), None)
        if next_waypoint is None:
            return TrajectoryState(t, position=self.waypoints[-1].position, yaw=self.waypoints[-1].yaw)

        # Else find the waypoints we're in between right now!
        prev_waypoint = self.waypoints[self.waypoints.index(next_waypoint) - 1]

        # And now that you have prev_waypoint and next_waypoint - generate a trajectory for a time somewhere in between!
        raise NotImplementedError()

        return TrajectoryState(t, position=position, yaw=yaw)

NameError: name 'TrajectoryBase' is not defined

In [ ]:
# NOTE(roy) this just writes an R now. Make it write something fun!
trajectory = WaypointTrajectory([
    Waypoint(0.0, position=np.zeros(3)),
    Waypoint(2.0, position=np.array([0, 0, 1])),
    Waypoint(4.0, position=np.array([0, 0.5, 1])),
    Waypoint(6.0, position=np.array([0, 0.5, 0.5])),
    Waypoint(8.0, position=np.array([0, 0.0, 0.5])),
    Waypoint(10.0, position=np.array([0, 0.5, 0.0])),
])

### Running the simulator

In [ ]:
from quadrotor.simulator import SimulatorBase, SimulatorState

# This might be slow to run the first time!
from quadrotor.renderer import animate_k3d

In [ ]:
def run_and_render(initial_state: QuadrotorState, t_total: int = 2.0):
    sim = SimulatorBase(
        dt=0.005,
        dynamics=FullQuadrotorDynamics(),
        controller=Controller(),
        trajectory=JumpTrajectory(),
        initial_state=initial_state,
        t_total=t_total,
    )
    
    # Run the simulator
    output = sim.simulate()

    # K3D renderer
    plot = animate_k3d(output)
    
    plot.display()
    plot.start_auto_play()

In [ ]:
# NOTE(roy) play with this variable and see what happens!
initial_state = QuadrotorState(
    position=np.zeros(3),
    velocity=np.array([0, 0, 0]),
    orientation=Rot3.from_yaw_pitch_roll(0, 0, 0),
    angular_velocity=np.zeros(3),
)

run_and_render(initial_state, 10.0)

### Looking at the controller behavior
If all is well, the quadrotor should follow the trajectory!

Try some experiments:
- Can you make the controller more/less damped and faster/slower to respond?
- Can you write a trajectory that has feasible velocities/positions?
    - Hint: the trajectory that's currently there is infeasible, why?

### Commit your updated notebook to your fork!